In [84]:
import pandas as pd
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Union
import statsmodels.api as sm

In [19]:
all_data = pd.read_csv('../data/all_data.csv').drop('Unnamed: 0', axis=1)

In [3]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA

# Code

In [4]:
def calculate_posterior_cov(
    factor_list: list=['factor_1', 'factor_2'],
    prior_cov: pd.DataFrame=pd.DataFrame(
        {'factor_1': [1, 1.1], 'factor_2': [1.1, 2]},
        index=['factor_1', 'factor_2']
    ),
    est_var=[1.1, 3],
):
    """
    склеиваем оценки дисперсий факторов и их ковариации
    """
    assert list(prior_cov.columns)==list(prior_cov.index), 'unexpected corr matrix index'

    LEN = len(factor_list)

    # создаем матрицу и добавляем оценки дисперсий факторов 
    res_cov = np.zeros(shape=(LEN, LEN)) + np.diag(est_var)

    # убираем лишнее и отсортировываем матрицу
    prior_cov = prior_cov.loc[factor_list, factor_list].to_numpy()

    # оставляем только ковариации
    only_cov = prior_cov - np.diag(np.diagonal(prior_cov))

    return res_cov + only_cov

In [5]:
import scipy.stats as sps

def sample_multivariate_normal(
    factor_list: list=['factor_1', 'factor_2'],
    prior_cov: pd.DataFrame=pd.DataFrame(
        {'factor_1': [1, 1.1], 'factor_2': [1.1, 2]},
        index=['factor_1', 'factor_2']
    ),
    est_var=[1.1, 3],
    est_mean: list=[10, 15],
    size=100,
    return_array=True,
):
    """
    В функцию подаем имена риск-факторов, средние и дисперсии в одном порядке!
    """
    posterior_cov = calculate_posterior_cov(factor_list, prior_cov, est_var)
    
    sample = sps.multivariate_normal(
        mean=est_mean,
        cov=posterior_cov,
    ).rvs(size=size)

    return sample if return_array else pd.DataFrame(sample, factor_list)

In [38]:
import math

def calculate_simulation_probability(
    factor_list: list,
    series_list: List[np.array],
    prior_cov: pd.DataFrame,
    est_var: List[np.array],
    est_mean: List[np.array],
    logging=False,
    logreturn=True,
):  
    """
    Returns smth similar to Log Likelihood s. t. Multivariate distribution
    using models individual forecasts and historical covariances
    """
    
    assert len(factor_list) == len(series_list), 'check number of factors'
    assert len(set(len(array) for array in series_list)) == 1, 'check num of days in provided series'

    prior_cov = prior_cov.loc[factor_list, factor_list]
    
    series_matrix = np.array(series_list).T
    est_var_matrix = np.array(est_var).T
    est_mean_matrix = np.array(est_mean).T

    LENGTH = series_matrix.shape[0]
    prob_list = [-1] * LENGTH
    
    for step in range(LENGTH):

        posterior_cov = calculate_posterior_cov(
            factor_list,
            prior_cov,
            est_var_matrix[step, :]
        )

        dist = sps.multivariate_normal(
            mean=est_mean_matrix[step, :],
            cov=posterior_cov
        )

        prob_list[step] = dist.pdf(series_matrix[step, :])

    if logging:
        print(prob_list)
        
    return np.sum(np.log(prob_list)) if logreturn else np.prod(prob_list)

In [71]:
from itertools import product
import warnings
from tqdm import tqdm


def find_last_acf_sign_lag(ts, drop_first = True):
    acf, ci = sm.tsa.acf(ts, alpha=0.01)
    first_zero, second_zero = 0, 0
    for l in range(1, len(ci)):
        if (0 >= ci[l][0] and 0 <= ci[l][1]):
            if first_zero == 0:
                first_zero = l - 1
            else: 
                second_zero = l - 1
                break
    return first_zero
    
def find_last_pacf_sign_lag(ts, drop_first = True):
    acf, ci = sm.tsa.pacf(ts, alpha=0.01)
    first_zero, second_zero = 0, 0
    for l in range(1, len(ci)):
        if (0 >= ci[l][0] and 0 <= ci[l][1]):
            if first_zero ==0:
                first_zero = l - 1
            else: 
                second_zero = l - 1
                break
    return first_zero   


def fit_best_sarima_model(
    series: pd.Series, 
    seasonal = 0,
    ps = None,
    d = None,
    qs = None,
    Ps = None,
    D= None,
    Qs = None,
):
    max_params = [
        find_last_acf_sign_lag(series.diff().dropna()),
        find_last_pacf_sign_lag(series.diff().dropna()), 
    1, 1]
    
    # print(max_params)
    ps = range(0, max_params[0] + 1) if not ps else ps
    d = [1] if not d else d
    qs = range(0, max_params[1] + 1) if not qs else qs
    Ps = range(0, max_params[2]) if not Ps else Ps
    D = [1] if not D else D
    Qs = range(0, max_params[3]) if not Qs else Qs
    
    grid = [ps, d, qs]

    if seasonal:
        grid += [Ps, D, Qs]
    
    parameters_list = list(product(*grid))
    results = []
    best_aic = float("inf")
    warnings.filterwarnings('ignore')

    
    for param in parameters_list:
        #try except нужен, потому что на некоторых наборах параметров модель не обучается
        order_list = (*param[:3],)
        seasonal_list = (*param[3:7],) if seasonal else (0, 0, 0, 0)
        
        try:
            model_sm = sm.tsa.SARIMAX(
                series,
                order=order_list, 
                seasonal_order=seasonal_list,
            ).fit(disp=-1)
            
        #выводим параметры, на которых модель не обучается и переходим к следующему набору
        except ValueError:
            print('wrong parameters:', param)
            continue
        aic = model_sm.aic
        
        #сохраняем лучшую модель, aic, параметры
        if aic < best_aic:
            best_model = model_sm
            best_aic = aic
            best_param = param
            
        results.append([param, model_sm.aic])
    
    return best_model, best_param

In [218]:
# (find_last_acf_sign_lag(train['aluminum'].diff().dropna()),
# find_last_pacf_sign_lag(train['aluminum'].diff().dropna()))

In [65]:
# plot_pacf(train['aluminum'].diff().dropna())

# Lets check

In [181]:
data = all_data.set_index('date')

In [182]:
train = data[data.index < '2023-12-01']

In [183]:
factor_list = ['brent', 'eur_rub', 'aluminum']

In [184]:
model_dict = {}
param_dict = {}

for f in tqdm(factor_list):

    model_dict[f], param_dict[f] = fit_best_sarima_model(train[f], ps=range(0, 6), qs=(0, 6))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.60s/it]


In [185]:
param_dict

{'brent': (2, 1, 0), 'eur_rub': (2, 1, 6), 'aluminum': (1, 1, 6)}

# 1 step

In [188]:
cov_matrix = train[factor_list].corr()

In [189]:
one_step_means = []
one_step_var = []

for name in factor_list:

    pred = model_dict[name].get_forecast(steps=1)

    one_step_means.append(pred.predicted_mean.values[0])
    one_step_var.append(pred.var_pred_mean.values[0])

In [190]:
sample = sample_multivariate_normal(
    factor_list=factor_list,
    prior_cov=cov_matrix,
    est_mean=one_step_means,
    est_var=one_step_var,size=5000        
)

sample.mean(axis=0)

array([  80.68683012,   97.79079649, 2196.84175808])

# N steps 

Можно конечно мб какую-то векторную авторегрессию сюда накрутить, но имхо оверкилл

In [191]:
STEPS = 10

means_series = []
var_series = []

for name in factor_list:

    pred = model_dict[name].get_forecast(steps=STEPS)

    means_series.append(pred.predicted_mean.values)
    var_series.append(pred.var_pred_mean.values)

Потестим фукцию для "вероятности" траектории

In [192]:
# нужен численный индекс последней записи на трейне +1
ANCHOR = train['brent'].shape[0]

In [207]:
simulation_2 = []

for name in factor_list:

    simulation_2.append(
        model_dict[name].simulate(nsimulations=STEPS,anchor=ANCHOR).values
    )

In [212]:
p = calculate_simulation_probability(
    factor_list,
    simulation_1,
    cov_matrix,
    var_series,
    means_series,
    logging=False,
    logreturn=True
)

In [228]:
def simulated_series_selection(
    factor_list: list,
    simulations: List[List[np.array]], # M iterations for N days for K factors
    prior_cov: pd.DataFrame,
    est_var: List[np.array],
    est_mean: List[np.array],
    threshold=None,
    quantile=0.05,
):
    
    NUM_SIM = len(simulations)
    probs = [0] * NUM_SIM
    
    for sim in range(NUM_SIM):
        
        probs[sim] = calculate_simulation_probability(
            factor_list,
            simulations[sim],
            prior_cov,
            est_var,
            est_mean,
            logging=False,
            logreturn=True
        )

    threshold = np.quantile(probs, quantile)
    
    for sim in range(NUM_SIM):
        if probs[sim] >= threshold:
            yield simulations[sim]
        
    

In [230]:
simulated_series_selection(
    factor_list,
    [simulation_1, simulation_1, simulation_2],
    cov_matrix,
    var_series,
    means_series,
).__next__()

[array([81.73161585, 81.70051407, 80.28470491, 82.61922812, 83.80273262,
        84.204375  , 85.63801739, 83.16580482, 85.02340704, 86.72693855]),
 array([100.52685996, 101.8409494 , 102.19929193, 101.35100257,
         99.08673198,  99.13833293,  97.38063755,  95.5635618 ,
         96.5623006 ,  98.89317195]),
 array([2168.98520265, 2161.99708466, 2156.79573212, 2239.41930567,
        2176.13973835, 2149.47969647, 2132.97684705, 2207.33658065,
        2199.90547324, 2140.07837643])]

In [256]:
sample_multivariate_normal()

In [231]:
# all_data.set_index('date').cov()['aluminum']['brent']